Author: Enrique Forero

Date: 2021-09-15

# ✅ Save and load python interpreter sessions

In [ ]:
# https://stackoverflow.com/questions/34342155/how-to-pickle-or-store-jupyter-ipython-notebook-session-for-later
pip install dill

In [ ]:
import dill
# Save a Notebook Session
dill.dump_session('notebook_env.db')

In [ ]:
import dill
# Restore a NoteBook Session
dill.load_session('notebook_env.db')

# ✅ Setting up the environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Indicando la ruta de la carpeta donde están los archivos. 
%cd '/content/drive/MyDrive/ProColombia/RUES'
%ls 

/content/drive/MyDrive/ProColombia/RUES
 2021-09-15_EXT-DBA-357-RUES-TICKET-8352.zip
'2021-09-24 NIT y Empresas.xlsx'
'ADJUST DATA TYPES RUES.xlsx'
'Cruce RUES - Exportaciones.ipynb'
 data_CSV.csv
 df_dtypes.xlsx


# ✅ Load data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
%%time
df_nits = pd.read_excel("2021-09-24 NIT y Empresas.xlsx")

CPU times: user 1.67 s, sys: 20.8 ms, total: 1.69 s
Wall time: 1.75 s


In [ ]:
df_nits.shape

(35893, 6)

In [ ]:
df_nits.head()

,NIT,RAZON_SOCIAL,CONTEO_A
0,01D100827,STEFAN FLORICA STEFANESCU ...,1
1,05AE80266,DE FRANCONY GERARD GILBERT ...,1
2,06AL24912,BOUDET MADELEINE MATHILDE ANDREE ...,1
3,06BA16405,FEIGELSON JOHANSSON ERIC WILLIAM ...,1
4,06PC00615,MACIA DURAN VANESSA ...,1


In [ ]:
df_nits.dtypes

NIT             object
RAZON_SOCIAL    object
CONTEO_A         int64
dtype: object

In [ ]:
%%time 
# LLega a ocupar 8GB mientras procesa la información. Al final queda de 4.31GB. 
df = pd.read_csv('2021-09-15_EXT-DBA-357-RUES-TICKET-8352.zip', encoding='latin-1', sep="|", error_bad_lines=False, low_memory=False)

CPU times: user 1min 38s, sys: 49.9 s, total: 2min 28s
Wall time: 2min 28s


In [ ]:
df.head()

,Cód Cámara,Matricula,Razón Social,Clase Identificación,Numero de Identificación,Digito de Verificación,Municipio Comercial,Dirección Comercial,Tel. Comercial 1,Tel. Comercial 2,Tel. Comercial 3,Correo Electrónico Comercial,Municipio Fiscal,Dirección Fiscal,Tel. Fiscal 1,Tel. Fiscal 2,Tel. Fiscal 3,Correo Electrónico Fiscal,Ultimo Año Renovado,Fecha Matricula,Fecha Renovación,Estado,Organización Jurídica,Categoría Matricula,CIIU Principal,CIIU Secundario,ciiu 3,ciiu 4,Empleados,Num Identificacion Representante Legal,Representante Legal,Fecha Actualizacion RUES
0,1.0,1,ARIAS MARIN SANTIAGO,CEDULA DE CIUDADANIA,1239941,NaN,63001,CR. 24 # 30-58,NaN,NaN,NaN,NaN,63001,CR. 24 # 30-58,NaN,NaN,NaN,NaN,1972,19720104,19720104,CANCELADA,PERSONA NATURAL,PERSONA NATURAL,9999,NaN,NaN,NaN,1.0,NaN,NaN,20210213.0
1,1.0,3,RIVERA OCHOA JOSE,CEDULA DE CIUDADANIA,1348019,NaN,63001,CARRERA 18 #30-16,NaN,NaN,NaN,NaN,63001,CARRERA 18 #30-16,NaN,NaN,NaN,NaN,0,19720104,,CANCELADA,PERSONA NATURAL,PERSONA NATURAL,9999,NaN,NaN,NaN,0.0,NaN,NaN,20201113.0
2,1.0,6,MARIN HOYOS JORGE,CEDULA DE CIUDADANIA,1275176,0,63001,CARRERA 20 #14-18,NaN,NaN,NaN,NaN,63001,CARRERA 20 #14-18,NaN,NaN,NaN,NaN,1980,19720104,19800108,CANCELADA,PERSONA NATURAL,PERSONA NATURAL,9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,7,ORTIZ MORALES BERNARDO ANTONIO,CEDULA DE CIUDADANIA,1238721,0,63001,CALLE 31 # 19-06,NaN,NaN,NaN,NaN,63001,CALLE 31 # 19-06,NaN,NaN,NaN,NaN,1994,19720104,19940131,CANCELADA,PERSONA NATURAL,PERSONA NATURAL,9999,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,10,DUQUE GALVIS EFRAIM,CEDULA DE CIUDADANIA,1268028,NaN,63130,CALLE 38 # 26-16,432628,NaN,NaN,NaN,63130,CALLE 38 # 26-16,NaN,NaN,NaN,NaN,0,19720104,,CANCELADA,PERSONA NATURAL,PERSONA NATURAL,9999,NaN,NaN,NaN,0.0,NaN,NaN,20201112.0


In [ ]:
df.shape

(7310903, 32)

In [ ]:
df.dtypes

Cód Cámara                                float64
Matricula                                   int64
Razón Social                               object
Clase Identificación                       object
Numero de Identificación                   object
Digito de Verificación                     object
Municipio Comercial                        object
Dirección Comercial                        object
Tel. Comercial 1                           object
Tel. Comercial 2                           object
Tel. Comercial 3                           object
Correo Electrónico Comercial               object
Municipio Fiscal                           object
Dirección Fiscal                           object
Tel. Fiscal 1                              object
Tel. Fiscal 2                              object
Tel. Fiscal 3                              object
Correo Electrónico Fiscal                  object
Ultimo Año Renovado                        object
Fecha Matricula                            object


# ✅ Estandarizar NIT de base exportaciones

In [ ]:
df_nits.columns

Index(['NIT', 'RAZON_SOCIAL', 'CONTEO_A'], dtype='object')

In [ ]:
# Utilizamos la función lamba que me permite iterar en cada fila. A cada valor de las celdas, lo nombro como "x"
# Como son números, primero debo convertirlos en texto utilizando la función srt(), se hace sobre la variable "x" 
df_nits['Len_nit'] = df_nits['NIT'].apply(lambda x: len(str(x)))

In [ ]:
# Verifico la cantidad de dígitos en cada posición. Todas deberían ser 10. De lo contrario, debo hacer ajustes.
df_nits['Len_nit'].unique()

array([ 9, 11,  8, 10,  7, 12,  6,  5,  3])

In [ ]:
%%time
# Check how many companies are by Len_Posicion. 
df_temp = pd.pivot_table(df_nits, values = "CONTEO_A" , index = ["Len_nit"], aggfunc=np.sum).reset_index() 
print(df_temp.shape)

(9, 2)
CPU times: user 21.4 ms, sys: 7.76 ms, total: 29.2 ms
Wall time: 82.5 ms


In [ ]:
df_temp

,Len_nit,CONTEO_A
0,3,2
1,5,2
2,6,19
3,7,2966
4,8,1120
5,9,8385
6,10,21711
7,11,1674
8,12,14


In [ ]:
# Creo función que me permite incluir un cero al inicio de cada posición que es de sólo 9 dígitos. 
def funcion_ajuste( nit , Len_nit  ):
    if ( Len_nit == 9 ) :      
        nit = "'0" + str(nit)
    elif ( Len_nit == 8 ) :  
        nit = "'00" + str(nit)
    elif ( Len_nit == 7 ) :  
        nit = "'000" + str(nit)
    else:
        nit = "'" + str(nit)

    return nit

In [ ]:
%%time
# Wall time: 1min 5s 
# Este proceso demanda varios recursos de memoria RAM. 
df_nits['NIT_Modificado'] = df_nits.apply( lambda x: funcion_ajuste( x['NIT'] , x['Len_nit']), axis=1 )

CPU times: user 472 ms, sys: 5.91 ms, total: 478 ms
Wall time: 498 ms


In [ ]:
# Filtro los NITS que son de 11 dígitos para asegurarnos que quedaron de 11 al incluirle un cero
df_nits[ (df_nits['Len_nit']==11) ][['NIT','Len_nit', 'NIT_Modificado']]

,NIT,Len_nit,NIT_Modificado
23,10004103352,11,'10004103352
25,10009433541,11,'10009433541
26,10012695800,11,'10012695800
27,10012749416,11,'10012749416
28,10013114041,11,'10013114041
...,...,...,...
35466,SWE27001557,11,'SWE27001557
35467,SWE27103360,11,'SWE27103360
35468,SWE27105351,11,'SWE27105351
35469,SWE97174683,11,'SWE97174683


## 🟠 Extraer los NITS a 10 dígitos

In [ ]:
# Extraigo los primeros 10 caracteres. De esta forma dejamos el código de verificación por fuera 
df_nits['NIT_10'] = df_nits['NIT_Modificado'].str[:10]

In [ ]:
# Verifico resultados. 
df_nits[ (df_nits['Len_nit']==11) ][['NIT','Len_nit', 'NIT_Modificado', 'NIT_10']]

,NIT,Len_nit,NIT_Modificado,NIT_10
23,10004103352,11,'10004103352,'100041033
25,10009433541,11,'10009433541,'100094335
26,10012695800,11,'10012695800,'100126958
27,10012749416,11,'10012749416,'100127494
28,10013114041,11,'10013114041,'100131140
...,...,...,...,...
35466,SWE27001557,11,'SWE27001557,'SWE270015
35467,SWE27103360,11,'SWE27103360,'SWE271033
35468,SWE27105351,11,'SWE27105351,'SWE271053
35469,SWE97174683,11,'SWE97174683,'SWE971746


In [ ]:
# Verifico resultados. 
df_nits[ (df_nits['Len_nit']==9) ][['NIT','Len_nit', 'NIT_Modificado', 'NIT_10']]

,NIT,Len_nit,NIT_Modificado,NIT_10
0,01D100827,9,'001D100827,'001D10082
1,05AE80266,9,'005AE80266,'005AE8026
2,06AL24912,9,'006AL24912,'006AL2491
3,06BA16405,9,'006BA16405,'006BA1640
4,06PC00615,9,'006PC00615,'006PC0061
...,...,...,...,...
35792,YA9366299,9,'0YA9366299,'0YA936629
35793,YB0525994,9,'0YB0525994,'0YB052599
35795,YB1272137,9,'0YB1272137,'0YB127213
35796,YB1887153,9,'0YB1887153,'0YB188715


In [ ]:
df_nits.shape

(35893, 6)

# ✅ Estandarizar NIT de RUES

In [ ]:
df.columns

Index(['Cód Cámara', 'Matricula', 'Razón Social', 'Clase Identificación',
       'Numero de Identificación', 'Digito de Verificación',
       'Municipio Comercial', 'Dirección Comercial', 'Tel. Comercial 1',
       'Tel. Comercial 2', 'Tel. Comercial 3', 'Correo Electrónico Comercial',
       'Municipio Fiscal', 'Dirección Fiscal', 'Tel. Fiscal 1',
       'Tel. Fiscal 2', 'Tel. Fiscal 3', 'Correo Electrónico Fiscal',
       'Ultimo Año Renovado', 'Fecha Matricula', 'Fecha Renovación', 'Estado',
       'Organización Jurídica', 'Categoría Matricula', 'CIIU Principal',
       'CIIU Secundario', 'ciiu 3', 'ciiu 4', 'Empleados',
       'Num Identificacion Representante Legal', 'Representante Legal',
       'Fecha Actualizacion RUES'],
      dtype='object')

In [ ]:
# Utilizamos la función lamba que me permite iterar en cada fila. A cada valor de las celdas, lo nombro como "x"
# Como son números, primero debo convertirlos en texto utilizando la función srt(), se hace sobre la variable "x" 
df['Len_nit'] = df['Numero de Identificación'].apply(lambda x: len(str(x)))

In [ ]:
# Verifico la cantidad de dígitos en cada posición. Todas deberían ser 10. De lo contrario, debo hacer ajustes.
df['Len_nit'].unique()

array([ 7,  8,  6,  9,  3,  1, 11,  4,  5,  2, 10, 14, 12, 13])

In [ ]:
df['CONTEO_B'] = 1

In [ ]:
%%time
# Check how many companies are by Len_Posicion. 
df_temp = pd.pivot_table(df, values = "CONTEO_B" , index = ["Len_nit"], aggfunc=np.sum).reset_index() 
print(df_temp.shape)

(14, 2)
CPU times: user 190 ms, sys: 35.3 ms, total: 225 ms
Wall time: 227 ms


In [ ]:
df_temp

,Len_nit,CONTEO_B
0,1,15725
1,2,1334
2,3,6792
3,4,6512
4,5,7986
5,6,71460
6,7,692843
7,8,3866041
8,9,1664313
9,10,934272


In [ ]:
# Creo función que me permite incluir un cero al inicio de cada posición que es de sólo 9 dígitos. 
# Como este no tiene el código de verificación, se le coloca un cero menos a los NITS de las empresas exportadoras. 
def funcion_ajuste( nit , Len_nit  ):
    if ( Len_nit == 9 ) :      
        nit = "'" + str(nit)
    elif ( Len_nit == 8 ) :  
        nit = "'0" + str(nit)
    elif ( Len_nit == 7 ) :  
        nit = "'00" + str(nit)
    else:
        nit = "'" + str(nit)

    return nit

In [ ]:
%%time
# Es necesario fraccionar el proceso para no ocupar toda la memoria
# Wall time: 1min 5s 
# Este proceso demanda varios recursos de memoria RAM. Por lo tanto, se hará por partes
df[:1000000]['NIT_Modificado'] = df[:1000000].apply( lambda x: funcion_ajuste( x['Numero de Identificación'] , x['Len_nit']), axis=1 )

CPU times: user 12 s, sys: 713 ms, total: 12.7 s
Wall time: 12.8 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
%%time
df[1000000:2000000]['NIT_Modificado'] = df[1000000:2000000].apply( lambda x: funcion_ajuste( x['Numero de Identificación'] , x['Len_nit']), axis=1 )

CPU times: user 12.4 s, sys: 207 ms, total: 12.6 s
Wall time: 12.6 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
%%time
df[2000000:3000000]['NIT_Modificado'] = df[2000000:3000000].apply( lambda x: funcion_ajuste( x['Numero de Identificación'] , x['Len_nit']), axis=1 )

CPU times: user 12.4 s, sys: 222 ms, total: 12.6 s
Wall time: 12.6 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
%%time
df[3000000:5000000]['NIT_Modificado'] = df[3000000:5000000].apply( lambda x: funcion_ajuste( x['Numero de Identificación'] , x['Len_nit']), axis=1 )

CPU times: user 24.1 s, sys: 1.4 s, total: 25.5 s
Wall time: 25.6 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
%%time
df[5000000:]['NIT_Modificado'] = df[5000000:].apply( lambda x: funcion_ajuste( x['Numero de Identificación'] , x['Len_nit']), axis=1 )

CPU times: user 28 s, sys: 558 ms, total: 28.6 s
Wall time: 28.6 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.shape

(7310903, 35)

In [ ]:
df.head(5)

In [ ]:
# Filtro los NITS que son de 11 dígitos para asegurarnos que quedaron de 11 al incluirle un cero
df[ (df['Len_nit']==11) ][['Numero de Identificación','Len_nit', 'NIT_Modificado']]

,Numero de Identificación,Len_nit,NIT_Modificado
106,50000000216,11,'50000000216
435,50000001802,11,'50000001802
667,50000002342,11,'50000002342
767,50000002586,11,'50000002586
907,50000002945,11,'50000002945
...,...,...,...
7310178,50000500120,11,'50000500120
7310179,50000500121,11,'50000500121
7310492,50000500463,11,'50000500463
7310495,50000500466,11,'50000500466


## 🟠 Extraer los NITS a 10 dígitos

In [ ]:
%%time 
# Extraigo los primeros 9 caracteres. 
df['NIT_10'] = df['NIT_Modificado'].str[:10]

In [ ]:
# Verifico resultados. 
df[ (df['Len_nit']==11) ][['Numero de Identificación','Len_nit', 'NIT_Modificado', 'NIT_10']]

,Numero de Identificación,Len_nit,NIT_Modificado,NIT_10
106,50000000216,11,'50000000216,'500000002
435,50000001802,11,'50000001802,'500000018
667,50000002342,11,'50000002342,'500000023
767,50000002586,11,'50000002586,'500000025
907,50000002945,11,'50000002945,'500000029
...,...,...,...,...
7310178,50000500120,11,'50000500120,'500005001
7310179,50000500121,11,'50000500121,'500005001
7310492,50000500463,11,'50000500463,'500005004
7310495,50000500466,11,'50000500466,'500005004


In [ ]:
# Verifico resultados. 
df[ (df['Len_nit']==9) ][['Numero de Identificación','Len_nit', 'NIT_Modificado', 'NIT_10']]

,Numero de Identificación,Len_nit,NIT_Modificado,NIT_10
46,890001559,9,'890001559,'890001559
100,890000188,9,'890000188,'890000188
121,890000416,9,'890000416,'890000416
154,890002474,9,'890002474,'890002474
183,890000198,9,'890000198,'890000198
...,...,...,...,...
7310898,901486944,9,'901486944,'901486944
7310899,901488693,9,'901488693,'901488693
7310900,901488274,9,'901488274,'901488274
7310901,901488269,9,'901488269,'901488269


In [ ]:
df.shape

(7310903, 36)

# ✅ Hacer el BuscarV

In [ ]:
%whos DataFrame

Variable   Type         Data/Info
---------------------------------
df         DataFrame             Cód Cámara   Mat<...>310903 rows x 36 columns]
df_nits    DataFrame                 NIT  ...    <...>n[35893 rows x 6 columns]
df_temp    DataFrame        Len_nit  CONTEO_B\n0 <...>82\n13       14      2822


In [ ]:
df.columns

Index(['Cód Cámara', 'Matricula', 'Razón Social', 'Clase Identificación',
       'Numero de Identificación', 'Digito de Verificación',
       'Municipio Comercial', 'Dirección Comercial', 'Tel. Comercial 1',
       'Tel. Comercial 2', 'Tel. Comercial 3', 'Correo Electrónico Comercial',
       'Municipio Fiscal', 'Dirección Fiscal', 'Tel. Fiscal 1',
       'Tel. Fiscal 2', 'Tel. Fiscal 3', 'Correo Electrónico Fiscal',
       'Ultimo Año Renovado', 'Fecha Matricula', 'Fecha Renovación', 'Estado',
       'Organización Jurídica', 'Categoría Matricula', 'CIIU Principal',
       'CIIU Secundario', 'ciiu 3', 'ciiu 4', 'Empleados',
       'Num Identificacion Representante Legal', 'Representante Legal',
       'Fecha Actualizacion RUES', 'Len_nit', 'CONTEO_B', 'NIT_Modificado',
       'NIT_10'],
      dtype='object')

In [ ]:
%%time
# Check if there is any "NIT" associated with more than one "RAZ_SIAL", if so, we need to correct all the mistakes. 
df_temp2 = pd.pivot_table(df, values = "CONTEO_B" , index = ["Clase Identificación"], aggfunc=np.sum).reset_index() # columns = "count",
df_temp2 = df_temp2.sort_values(['CONTEO_B'], ascending=[False])
print(df_temp2.shape)

(14, 2)
CPU times: user 440 ms, sys: 7.74 ms, total: 447 ms
Wall time: 449 ms


In [ ]:
df_temp2.head()

,Clase Identificación,CONTEO_B
2,CEDULA DE CIUDADANIA,4966945
8,NIT,2228242
12,SIN IDENTIFICACION,83228
3,CEDULA DE EXTRANJERIA,20522
9,PASAPORTE,9243


In [ ]:
# Filtro clase 
# df[['NIT_10','Razón Social',"CONTEO_B",'Ultimo Año Renovado', 'Fecha Matricula', 'Fecha Renovación', 'Estado']]

In [ ]:
%%time 
# Merge df_NITS. Filtro la tabla para solo trabajar con la base RUES donde la Clase Identificación == NIT
df_NITS_Final = pd.merge( df_nits , df[ (df['Clase Identificación']=="NIT") ][['NIT_Modificado','NIT_10','Razón Social','Clase Identificación',"CONTEO_B",
                                                                               'Ultimo Año Renovado', 'Fecha Matricula', 'Fecha Renovación', 'Estado']] , how = 'left' , 
              left_on = [ 'NIT_10' ] , right_on = [ 'NIT_10' ])

CPU times: user 4.67 s, sys: 35.7 ms, total: 4.71 s
Wall time: 4.69 s


In [ ]:
df_nits.shape

(35893, 6)

In [ ]:
df_NITS_Final.shape

(40444, 14)

In [ ]:
df_NITS_Final.head()

,NIT,RAZON_SOCIAL,CONTEO_A,Len_nit,NIT_Modificado_x,NIT_10,NIT_Modificado_y,Razón Social,Clase Identificación,CONTEO_B,Ultimo Año Renovado,Fecha Matricula,Fecha Renovación,Estado
0,01D100827,STEFAN FLORICA STEFANESCU ...,1,9,'001D100827,'001D10082,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,05AE80266,DE FRANCONY GERARD GILBERT ...,1,9,'005AE80266,'005AE8026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,06AL24912,BOUDET MADELEINE MATHILDE ANDREE ...,1,9,'006AL24912,'006AL2491,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,06BA16405,FEIGELSON JOHANSSON ERIC WILLIAM ...,1,9,'006BA16405,'006BA1640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,06PC00615,MACIA DURAN VANESSA ...,1,9,'006PC00615,'006PC0061,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_NITS_Final = df_NITS_Final[ (df_NITS_Final['CONTEO_B']==1 ) ]

In [ ]:
df_NITS_Final.shape

(25404, 14)

In [ ]:
df_NITS_Final.head()

,NIT,RAZON_SOCIAL,CONTEO_A,Len_nit,NIT_Modificado_x,NIT_10,NIT_Modificado_y,Razón Social,Clase Identificación,CONTEO_B,Ultimo Año Renovado,Fecha Matricula,Fecha Renovación,Estado
81,10076651861,ALVAREZ CORREA WILMER DUBAN ...,1,11,'10076651861,'100766518,'1007665186,ALVAREZ CORREA WILMER DUBAN,NIT,1.0,2013,20130523,20130523,CANCELADA
82,10076651861,ALVAREZ CORREA WILMER DUBAN ...,1,11,'10076651861,'100766518,'1007665186,ALVAREZ CORREA WILMER DUBAN,NIT,1.0,2019,20170110,20190307,MATRÍCULA CANCELADA POR TRASLADO DE DOMICILIO
104,10101756101,ALONSO VILLAMIZAR MARIA ALEJANDRA ...,1,11,'10101756101,'101017561,'1010175614,CASTAÑEDA MALES SANDRA YANETH,NIT,1.0,2019,20170313,20190326,ACTIVA
116,10101913548,AGUILERA CORREA MARCO ANTONIO ...,1,11,'10101913548,'101019135,'1010191357,ESPAÑA LOSADA GILBERTO,NIT,1.0,2016,20140319,20161202,CANCELADA
118,10101959441,PEDROZA OSPINA CAMILO ALFONSO ...,1,11,'10101959441,'101019594,'1010195946,AMEZQUITA OROZCO LIZANDRO,NIT,1.0,2011,20110701,,CANCELADA


In [ ]:
# SUmando las observaciones que obtenemos
df_NITS_Final["CONTEO_B"].sum()

25404.0

In [ ]:
df_nits.columns

Index(['NIT', 'RAZON_SOCIAL', 'CONTEO_A', 'Len_nit', 'NIT_Modificado',
       'NIT_10'],
      dtype='object')

In [ ]:
df.columns

Index(['Cód Cámara', 'Matricula', 'Razón Social', 'Clase Identificación',
       'Numero de Identificación', 'Digito de Verificación',
       'Municipio Comercial', 'Dirección Comercial', 'Tel. Comercial 1',
       'Tel. Comercial 2', 'Tel. Comercial 3', 'Correo Electrónico Comercial',
       'Municipio Fiscal', 'Dirección Fiscal', 'Tel. Fiscal 1',
       'Tel. Fiscal 2', 'Tel. Fiscal 3', 'Correo Electrónico Fiscal',
       'Ultimo Año Renovado', 'Fecha Matricula', 'Fecha Renovación', 'Estado',
       'Organización Jurídica', 'Categoría Matricula', 'CIIU Principal',
       'CIIU Secundario', 'ciiu 3', 'ciiu 4', 'Empleados',
       'Num Identificacion Representante Legal', 'Representante Legal',
       'Fecha Actualizacion RUES', 'Len_nit', 'CONTEO_B', 'NIT_Modificado',
       'NIT_10'],
      dtype='object')

Falta hacer una comparación entre los NITS completos.

LO que hice fue utilizar excel. Quitar los espacios de la razón social en la base exportaciones y luego comparar con los nombres del RUES. SI(Nombre_EXpo=Nombre_RUES,1,0)

# ✅ Exportar base de datos

In [ ]:
%%time
# Toma 1min 27s
df_NITS_Final.to_csv( "df_NITS_Final.csv" , encoding='utf-8-sig', index = False) 
df_NITS_Final.to_excel('df_NITS_Final.xlsx')

CPU times: user 10.5 s, sys: 24.1 ms, total: 10.6 s
Wall time: 10.6 s
